In [1]:
from Scripts.core import read_sbml_into_cobra_model
from Scripts.utils import ReconstructionTool, get_reactions, get_genes, get_metabolites, calculate_quality_metrics
import pandas as pd
import openpyxl

'''
import sys, importlib
importlib.reload(sys.modules['Scripts.utils'])
from Scripts.utils import ReconstructionTool, get_kegg_reactions, get_genes, get_kegg_metabolites
'''

"\nimport sys, importlib\nimportlib.reload(sys.modules['Scripts.utils'])\nfrom Scripts.utils import ReconstructionTool, get_kegg_reactions, get_genes, get_kegg_metabolites\n"

In [3]:
merlin_blast_model = read_sbml_into_cobra_model(
    file_path = "../Models/Merlin-BA/merlin_model.xml",
    database_version = "kegg",
    reconstruction_tool = ReconstructionTool.MERLIN.value)

carveme_model = read_sbml_into_cobra_model(
    file_path = "../Models/CarveMe/model_carveme.xml",
    database_version = "bigg",
    reconstruction_tool = ReconstructionTool.CARVEME.value)

kbase_model = read_sbml_into_cobra_model(
    file_path = "../Models/KBase/kbase_model.xml",
    database_version = "modelseed",
    reconstruction_tool = ReconstructionTool.MODELSEED.value)

merlin_bit_model = read_sbml_into_cobra_model(
    file_path = "../Models/Merlin-BIT/BIT_model.xml",
    database_version = "bigg",
    reconstruction_tool = ReconstructionTool.MERLIN.value)

aureme_model = read_sbml_into_cobra_model(
    file_path = "../Models/AuReMe/aureme_model3.xml",
    database_version = "bigg",
    reconstruction_tool = ReconstructionTool.AUREME.value)

1204725 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
1204725 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized
1204725 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


In [4]:
from Scripts.utils import get_reactions, get_metabolites, get_cross_reference_reactions, get_cross_reference_metabolites

models = {
    "merlin_blast": merlin_blast_model.model,
    "aureme": aureme_model.model,
    "carveme": carveme_model.model,
    "kbase": kbase_model.model,
    "merlin_bit": merlin_bit_model.model
}
func_ids = []
metanetx_ids = []
kegg_ids = []
reactions_conversion_df = pd.read_csv('Xrefs files/reactions-conversion.csv')
metabolites_conversion_df = pd.read_csv('Xrefs files/compounds-conversion.csv')

for name, xml in models.items():
    print(name)
    func_ids.append({
        'tool': name,
        'genes': ','.join(get_genes(xml, tool=name)),
        'reactions': ','.join(get_reactions(xml, tool=name)),
        'metabolites': ','.join(get_metabolites(xml, tool=name))
    })
    cross_reference_reactions = get_cross_reference_reactions(xml, reactions_conversion_df, tool=name)
    cross_reference_metabolites = get_cross_reference_metabolites(xml, metabolites_conversion_df, tool=name)
    metanetx_ids.append({
        'tool': name,
        'genes': ','.join(get_genes(xml, tool=name)),
        'reactions': ','.join(cross_reference_reactions[0]),
        'metabolites': ','.join(cross_reference_metabolites[0])
    })
    kegg_ids.append({
        'tool': name,
        'genes': ','.join(get_genes(xml, tool=name)),
        'reactions': ','.join(cross_reference_reactions[1]),
        'metabolites': ','.join(cross_reference_metabolites[1])
    })
pd.DataFrame(func_ids).to_csv('../Results/functional_ids.tsv', sep='\t', index=False)
pd.DataFrame(metanetx_ids).to_csv('../Results/metanetx_functional_ids.tsv', sep='\t', index=False)
pd.DataFrame(kegg_ids).to_csv('../Results/kegg_functional_ids.tsv', sep='\t', index=False)

merlin_blast
Found 722 reactions.
Found 382 reactions.
aureme
Found 400 reactions.
Found 645 reactions.
carveme
Found 2679 reactions.
Found 1583 reactions.
kbase
Found 491 reactions.
Found 538 reactions.
merlin_bit
Found 5159 reactions.
Found 4226 reactions.


In [5]:
formicicum = pd.read_csv('../Results/formicicum_uniprotinfo.tsv', sep='\t')
formicicum_genes = formicicum['Entry'].tolist()
metrics = []
for tool in ['merlin_blast', 'merlin_bit', 'carveme', 'kbase']:
    tool_uniprotinfo = pd.read_csv(f'../Results/{tool}_uniprotinfo.tsv', sep='\t')
    tool_genes = tool_uniprotinfo['Entry'].tolist()
    TPs = len([ide for ide in tool_genes if ide in formicicum_genes])
    FPs = len([ide for ide in tool_genes if ide not in formicicum_genes])
    FNs = len([ide for ide in formicicum_genes if ide not in tool_genes])
    metrics.append([tool, TPs, FPs, FNs] + list(calculate_quality_metrics(TPs, FPs, FNs)))
pd.DataFrame(metrics, columns=['tool', 'TPs', 'FPs', 'FNs', 'Precision', 'Recall', 'F1 score', 'Jaccard distance']).to_excel('../Results/quality_metrics.xlsx', index=False)


In [6]:
formicicum_reactions = open('../Results/formicicum_reactions.txt').read().split('\n')
func_ids_df = pd.read_csv('../Results/metanetx_functional_ids.tsv', sep='\t', index_col=0)

conversion = pd.read_csv('../Scripts/Xrefs files/reactions-conversion.csv')

conversion['External ID'].isin(formicicum_reactions).sum() #Verdadeiros

reactions = conversion[conversion['External ID'].isin(formicicum_reactions)]['Internal ID'].tolist()
print(reactions)

metrics = []
for tool in ['aureme', 'merlin_blast', 'merlin_bit', 'carveme', 'kbase']:
    model_reactions = func_ids_df.loc[tool,'reactions'].split(',')
    TPs = len([ide for ide in model_reactions if ide in reactions])
    FPs = len([ide for ide in model_reactions if ide not in reactions])
    FNs = len([ide for ide in reactions if ide not in model_reactions])
    print(TPs, FNs, FPs)
    metrics.append([tool, TPs, FPs, FNs] + list(calculate_quality_metrics(TPs, FPs, FNs)))
pd.DataFrame(metrics, columns=['tool', 'TPs', 'FPs', 'FNs', 'Precision', 'Recall', 'F1 score', 'Jaccard distance']).to_excel(
    '../Results/quality_metrics_reactions.xlsx', index=False)

['rxn00647', 'rxn34664', 'rxn34671', 'rxn00029', 'rxn00048', 'rxn00060', 'rxn00062', 'rxn00065', 'rxn00069', 'rxn00076', 'rxn00077', 'rxn00096', 'rxn00097', 'rxn00102', 'rxn00105', 'rxn00107', 'rxn00109', 'rxn00117', 'rxn00119', 'rxn00122', 'rxn00126', 'rxn00132', 'rxn00138', 'rxn00139', 'rxn00141', 'rxn00147', 'rxn00154', 'rxn00159', 'rxn00172', 'rxn00175', 'rxn00176', 'rxn00178', 'rxn00184', 'rxn00187', 'rxn00189', 'rxn00192', 'rxn00194', 'rxn00200', 'rxn00202', 'rxn00213', 'rxn00214', 'rxn00226', 'rxn00237', 'rxn00238', 'rxn00248', 'rxn00250', 'rxn00254', 'rxn00260', 'rxn00278', 'rxn00285', 'rxn00295', 'rxn00297', 'rxn00298', 'rxn00307', 'rxn00313', 'rxn00337', 'rxn00346', 'rxn00363', 'rxn00364', 'rxn00365', 'rxn00391', 'rxn00405', 'rxn00409', 'rxn00414', 'rxn00416', 'rxn00420', 'rxn00423', 'rxn00436', 'rxn00438', 'rxn00453', 'rxn00459', 'rxn00474', 'rxn00489', 'rxn00490', 'rxn00493', 'rxn00513', 'rxn00514', 'rxn00515', 'rxn00527', 'rxn00529', 'rxn00549', 'rxn00555', 'rxn00566', 'rx

In [10]:
formicicum_metabolites = open('../Results/formicicum_metabolites.txt').read().split('\n')
func_ids_df = pd.read_csv('../Results/metanetx_functional_ids.tsv', sep='\t', index_col=0)

conversion = pd.read_csv('../Scripts/Xrefs files/compounds-conversion.csv')

conversion['External ID'].isin(formicicum_metabolites).sum() #Verdadeiros

metabolites = conversion[conversion['External ID'].isin(formicicum_metabolites)]['Internal ID'].tolist()
mtx_metabolites = conversion[(conversion['External ID'].isin(metabolites)) & (conversion['Internal ID'].str.startswith('MNXM'))]['Internal ID'].tolist()
print(mtx_metabolites[:10])


metrics = []
for tool in ['aureme', 'merlin_blast', 'carveme', 'kbase', 'merlin_bit']:
    print(tool)
    model_metabolites = func_ids_df.loc[tool, 'metabolites'].split(',')
    print(model_metabolites[:10])

    TPs = len([ide for ide in model_metabolites if ide in mtx_metabolites])
    FPs = len([ide for ide in model_metabolites if ide not in mtx_metabolites])
    FNs = len([ide for ide in mtx_metabolites if ide not in model_metabolites])
    print(TPs, FNs, FPs)
    metrics.append([tool, TPs, FPs, FNs] + list(calculate_quality_metrics(TPs, FPs, FNs)))

pd.DataFrame(metrics, columns=['tool', 'TPs', 'FPs', 'FNs', 'Precision', 'Recall', 'F1 score', 'Jaccard distance']).to_excel(
    '../Results/quality_metrics_compounds.xlsx', index=False)

['MNXM1', 'MNXM10', 'MNXM100', 'MNXM100', 'MNXM10010', 'MNXM1002', 'MNXM10059', 'MNXM10066', 'MNXM100790', 'MNXM10129']
aureme
['cpd00001', 'cpd00001', 'cpd00002', 'cpd00002', 'cpd00003', 'cpd00003', 'cpd00004', 'cpd00004', 'cpd00005', 'cpd00005']
154 3915 491
merlin_blast
['cpd00001', 'cpd00002', 'cpd00003', 'cpd00004', 'cpd00005', 'cpd00006', 'cpd00007', 'cpd00008', 'cpd00009', 'cpd00010']
164 3910 218
carveme
['cpd00001', 'cpd00001', 'cpd00002', 'cpd00002', 'cpd00003', 'cpd00003', 'cpd00004', 'cpd00004', 'cpd00005', 'cpd00005']
411 3654 1172
kbase
['cpd00001', 'cpd00002', 'cpd00003', 'cpd00004', 'cpd00005', 'cpd00006', 'cpd00007', 'cpd00008', 'cpd00009', 'cpd00010']
200 3868 338
merlin_bit
['cpd00001', 'cpd00001', 'cpd00002', 'cpd00002', 'cpd00003', 'cpd00003', 'cpd00004', 'cpd00004', 'cpd00005', 'cpd00005']
720 3357 3506
